In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set()

# Импорт датасета и некоторые переменные с прошлого шага

In [2]:
raw_dataset = pd.read_csv("dataset.csv", sep=';', decimal=',', index_col=0)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
# Категориальные признаки
categorical_features = ["МАРКА", "ПРОФИЛЬ"]

# Вещественные признаки
numerical_features = ['t вып-обр', 't обработка',
       't под током', 't продувка', 'ПСН гр.', 'чист расход C',
       'чист расход Cr', 'чист расход Mn', 'чист расход Si', 'чист расход V',
       'температура первая', 'температура последняя', 'Ar (интенс.)',
       'N2 (интенс.)', 'эл. энергия (интенс.)', 'произв жидкая сталь',
       'произв количество обработок', 'произв  количество плавок',
       'произв количество плавок (цел)', 'расход газ Ar', 'расход газ  N2',
       'расход C пров.', 'сыпуч известь РП', 'сыпуч кварцит',
       'сыпуч кокс пыль УСТК', 'сыпуч  кокс. мелочь (сух.)',
       'сыпуч  кокс. мелочь КМ1', 'сыпуч  шпат плав.', 'ферспл CaC2',
       'ферспл FeMo', 'ферспл FeSi-75', 'ферспл FeV азот.', 'ферспл  FeV-80',
       'ферспл  Mn5Si65Al0.5', 'ферспл  Ni H1 пласт.', 'ферспл SiMn18',
       'ферспл  ферванит', 'ферспл  фх850А', 'эл. энергия',
       'химсталь первый Al_1', 'химсталь первый C_1', 'химсталь первый Cr_1',
       'химсталь первый Cu_1', 'химсталь первый Mn_1', 'химсталь первый Mo_1',
       'химсталь первый N_1', 'химсталь первый Ni_1', 'химсталь первый P_1',
       'химсталь первый S_1', 'химсталь первый Si_1', 'химсталь первый Ti_1',
       'химсталь первый V_1', 'химсталь последний Al', 'химсталь последний C',
       'химсталь последний Ca', 'химсталь последний Cr',
       'химсталь последний Cu', 'химсталь последний Mn',
       'химсталь последний Mo', 'химсталь последний N',
       'химсталь последний Ni', 'химсталь последний P', 'химсталь последний S',
       'химсталь последний Si', 'химсталь последний Ti',
       'химсталь последний V', 'химшлак первый Al2O3_1',
       'химшлак первый CaO_1', 'химшлак первый FeO_1', 'химшлак первый MgO_1',
       'химшлак первый MnO_1', 'химшлак первый R_1', 'химшлак первый SiO2_1',
       'химшлак последний Al2O3', 'химшлак последний CaO',
       'химшлак последний FeO', 'химшлак последний MgO',
       'химшлак последний MnO', 'химшлак последний R',
       'химшлак последний SiO2']

# Вещественные признаки с большим (>33%) количеством пропущенных значений
features_with_lots_of_nan = ['N2 (интенс.)', 'расход газ  N2', 'расход C пров.',
                             'сыпуч кварцит', 'сыпуч  кокс. мелочь (сух.)', 
                             'ферспл FeMo', 'ферспл FeSi-75', 'ферспл FeV азот.', 
                             'ферспл  Ni H1 пласт.', 'ферспл  ферванит']

# Обработка категориальных признаков

## Удаление ненужных признаков

Признаки "nplv" и "DT", очевидно, не несут в себе полезной информации - их можно удалить.

In [4]:
dataset = raw_dataset.drop(["nplv", "DT"], axis=1)

## Удаление редких признаков

Признаки, которые встречаются редко (<20%), можно объединить в одну категорию "rare"

In [5]:
def find_frequent_labels(df, var, rare_perc):
    """Ищет часто встречающиеся значения"""
    df = df.copy()
    
    tmp = df.groupby(var)[var].count() / len(df)
    
    return tmp[tmp > rare_perc].index


for feature in categorical_features:
    frequent_labels = find_frequent_labels(dataset, feature, 0.2)
    print(feature)
    print(frequent_labels)
    
    # заменяет редкие значения признаков на строку "rare"
    dataset[feature] = np.where(dataset[feature].isin(frequent_labels), dataset[feature], 'rare')

МАРКА
Index(['Э76ХФ'], dtype='object', name='МАРКА')
ПРОФИЛЬ
Index(['Р65'], dtype='object', name='ПРОФИЛЬ')


In [6]:
dataset["МАРКА"].value_counts()

Э76ХФ    4953
rare     2088
Name: МАРКА, dtype: int64

In [7]:
dataset["ПРОФИЛЬ"].value_counts()

Р65     6427
rare     614
Name: ПРОФИЛЬ, dtype: int64

# Обработка вещественных признаков

## Удаление признаков, в которых более 33% NaN
Список таких признаков выделен на предыдущем этапе

In [8]:
dataset = dataset.drop(features_with_lots_of_nan, axis=1)

## Удаление строк, в которых более 33% столбцов NaN

In [9]:
def list_rows_with_lots_of_nan(df):
    rows_list = []
    numbers_of_nan = df.isna().sum(axis=1)
    for index, count in enumerate(numbers_of_nan):
        if count > df.shape[1] / 3:
            rows_list.append(index)
    return rows_list

In [10]:
rows_with_lots_of_nan = list_rows_with_lots_of_nan(dataset)
print("Строк с более чем 33% пропусков: {}".format(len(rows_with_lots_of_nan)))

print("Число строк до удаления: {}".format(dataset.shape[0]))
dataset = dataset.drop(rows_with_lots_of_nan, axis=0)
print("Число строк после удаления: {}".format(dataset.shape[0]))

Строк с более чем 33% пропусков: 597
Число строк до удаления: 7041
Число строк после удаления: 6444


## Признаки с единственным уникальным значением
У некоторых признаков всего одно уникальное значение (помимо NaN). Это может означать одно из двух:
- Либо это бинарный категориальный признак, где 1 - наличие значения, 0 - отсутствие
- Либо этот признак не несет в себе никакой пользы.

На предыдущем этапе мы выделили два (три, но один уже удален из-за большого числа пропусков) таких пропуска - _'произв  количество плавок'_ и _'произв количество плавок (цел)'_, у обоих 597 (мало) пропусков, что позволяет откинуть вариант о бинарном признаке и просто удалить его.

In [11]:
def list_features_with_single_value(df):
    features_list = []
    for feature in df.columns:
        if df[feature].nunique() == 1:
            features_list.append(feature)
    return features_list

In [12]:
features_with_single_value = list_features_with_single_value(dataset)
print("Признаки с единственным значением: {}".format(features_with_single_value))

print("Было {} признаков".format(dataset.shape[1]))
dataset = dataset.drop(features_with_single_value, axis=1)
print("Стало {} признаков".format(dataset.shape[1]))

Признаки с единственным значением: ['произв  количество плавок', 'произв количество плавок (цел)']
Было 72 признаков
Стало 70 признаков
